Version Date: June 17, 2021

# Objectives
- Gain more experience in reading and modifying Python code in the Google Colab/Jupyter notebook environment
- Build machine learning multiclass classification models to predict whether a molecule contains a ketone, a carboxylic acid, or another type of carbonyl functional group using IR spectroscopy data

This notebook walks you through the process of carrying out this multiclass classification analysis but does not provide all the required code. You will add new code blocks to complete this notebook. The **BOLD ALL CAPS** text below indicates the places where you will need to write your own code.

# Getting Started
First you will need to import the required Python libraries for working with and visualizing the data, and for carrying out the machine learning.

**ADD A CODE BLOCK HERE TO IMPORT LIBRARIES**

# Get Data
Like in the binary classification task, you will need a training data set and a test data set:
- In this case, all molecules in both data sets contain a carbonyl group, so the three classes are: ketone, carboxylic acid, or other carbonyl group
- Unlike in the binary classification task, the training data is saved in three separate files: one with a label indicating whether or not a ketone is present, one with a label indicating whether or not a carboxylic acid is present, and one with a label indicating whether or not another carbonyl is present (i.e., whether neither ketone nor carboxylic acid are present)
- The test data set is stored in a single file with three columns columns corresponding to the labels for ketone, carboxylic acid, and other. 
- In all cases, the presence of a particular group is indicated by the label **1** and the absence by the label **0**

In [ ]:
# load the training data
ketone=pd.read_csv('https://raw.githubusercontent.com/elizabeththrall/MLforPChem/main/MLforvibspectroscopy/Data/multi_ketone.csv',index_col=0)
cbxl_acid=pd.read_csv('https://raw.githubusercontent.com/elizabeththrall/MLforPChem/main/MLforvibspectroscopy/Data/multi_cbxl_acid.csv',index_col=0)
other=pd.read_csv('https://raw.githubusercontent.com/elizabeththrall/MLforPChem/main/MLforvibspectroscopy/Data/multi_other.csv',index_col=0)
# load the test data
test=pd.read_csv('https://raw.githubusercontent.com/elizabeththrall/MLforPChem/main/MLforvibspectroscopy/Data/multi_test.csv',index_col=0)

**ADD A CODE BLOCK HERE TO DISPLAY THE CONTENTS OF THESE VARIABLES IN ORDER TO FAMILIARIZE YOURSELF WITH THEIR FORMAT**

# Data Preprocessing
As in the binary classification task, you will need to preprocess the data before carrying out the machine learning analysis.

## Normalization

In this section you will use min-max normalization to scale the intensities in each spectrum to run from 0 to 1

**ADD A CODE BLOCK HERE TO DEFINE THE df_normalize FUNCTION (HINT: IT CAN BE THE SAME AS IN THE BINARY CLASSIFICATION NOTEBOOK)**

**ADD A CODE BLOCK HERE TO USE THE df_normalize FUNCTION TO NORMALIZE THE TRAINING AND TEST DATA (HINT: BE SURE THAT YOU PROVIDE THE STARTING COLUMN FOR THE FREQUENCY DATA IN THE TEST DATA, SINCE IT IS DIFFERENT FROM THE DEFAULT VALUE FOR THE FUNCTION)**

## Apply Threshold
In this section you will apply a threshold to the training and test data to set all intensities below the threshold equal to 0.

**ADD A CODE BLOCK HERE TO PLOT THE SPECTRA OF SELECTED MOLECULES BEFORE THRESHOLDING**

**ADD A CODE BLOCK HERE TO SAVE A THRESHOLD VALUE IN A VARIABLE NAMED threshold**

**ADD A CODE BLOCK HERE TO DEFINE THE applyThreshold FUNCTION (HINT: IT CAN BE THE SAME AS IN THE BINARY CLASSIFICATION NOTEBOOK)**


**ADD A CODE BLOCK HERE TO USE THE applyThreshold FUNCTION TO THRESHOLD THE TRAINING AND TEST DATA (HINT: BE SURE THAT YOU PROVIDE THE STARTING COLUMN FOR THE FREQUENCY DATA IN THE TEST DATA, SINCE IT IS DIFFERENT FROM THE DEFAULT VALUE FOR THE FUNCTION)**

**ADD A CODE BLOCK HERE TO PLOT THE SPECTRA OF SELECTED MOLECULES AFTER THRESHOLDING**

## Split Attribute and Label
Now we need to separate the label from the attribute for the training and test data. Recall that the label tells us whether or not the molecule contains a ketone, carboxylic acid, or another carbonyl. The spectral intensities are the attributes.

Previously we saved the training data attributes in the variable **X** and the label in the variable **Y**. But now we have three different training sets, so we will save the attributes in three separate variables (**X_ketone**, **X_cbxl**, and **X_other**) and the labels in three separate variables (**Y_ketone**, **Y_cbxl**, and **Y_other**).

When you split the test data, save the attributes in the variable **X_test** and the labels in the variable **Y_test**, like before.

**ADD A CODE BLOCK HERE TO DEFINE THE splitXY FUNCTION (HINT: IT CAN BE THE SAME AS IN THE BINARY CLASSIFICATION NOTEBOOK)**

**ADD A CODE BLOCK HERE TO USE THE splitXY FUNCTION TO SPLIT THE ATTRIBUTE AND LABEL FOR THE TRAINING AND TEST DATA (HINT: BE SURE THAT YOU PROVIDE THE START AND END COLUMNS FOR THE LABELS AND THE FREQUENCY DATA IN THE TEST DATA SET, SINCE THEY ARE DIFFERENT FROM THE DEFAULT VALUES DEFINIED FOR THE FUNCTION)**

## Data Balancing
The three classes (ketone, carboxylic acid, and other carbonyl group) are imbalanced in the training data set. Thus we need to balance the data before training a machine learning model. You will use the synthetic minority oversampling technique (SMOTE) for this data balancing step. Because there are three sets of training data, you will need to use SMOTE on each one separately.

**ADD A CODE BLOCK HERE TO APPLY SMOTE TO EACH OF THE THREE TRAINING DATA SETS**



# Building Random Forest Machine Learning Models
You will use the Random Forest machine learning algorithm for this multiclass classification task. The first step is model training using the training data. Instead of training a single model, you will train three different models: one for the ketone training data, one for the carboxylic acid training data, and one for the other carbonyl training data. Store the trained models in the variables: **rf_ketone**, **rf_cbxl**, and **rf_other**.

**ADD A CODE BLOCK HERE TO TRAIN THE THREE RANDOM FOREST MODELS USING THE THREE TRAINING DATA SETS AND TO SAVE THE TRAINED MODELS IN THE VARIABLES LISTED ABOVE**

# Testing Machine Learning Models
Now you will use the trained machine learning models for label prediction on the test data set. You will use each of the the three trained models on the same test data. Store the predicted labels in the variables: **Y_k_pred** (for ketone), **Y_c_pred** (for carboxylic acid), and **Y_o_pred** (for other carbonyl group).

**ADD A CODE BLOCK HERE TO USE THE THREE TRAINED RANDOM FOREST MODELS TO PREDICT THE TEST DATA LABELS AND SAVE THE PREDICTED LABELS IN THE VARIABLES LISTED ABOVE**

Because each of these binary predictions is separate from the other two, it's possible that some molecules will either 1) be predicted to be a member of two different classes or 2) be predicted NOT to be a member of any class. While the first outcome could be correct (think about why!), the second outcome cannot be -- carbonyl-containining molecules must fall into one of the three categories (ketone, carboxylic acid, or other).

Run the code block below to display a table of the three predictions and look for any examples of these two outcomes.

In [ ]:
# get the molecule names
molName=test.iloc[:,0]
# create a DataFrame with the molecule names and the three prediction probabilities
d = {'molecules':molName,'ketone':Y_k_pred, 'carboxylic_acid':Y_c_pred,'other':Y_o_pred}
result = pd.DataFrame(data=d)
# get the truth for each molecule by looking at the labels in the test data set
result['truth']=test.iloc[:,2:5].idxmax(axis=1)
# display the table
result

To resolve these issues, we can use a different approach. Instead of having each model return a predicted label, we can instead obtain a metric related to the probability of a given label (i.e., how confident the model is in the prediction). We can then set the final predicted label as the label with the highest probability.

Run the code block below to get the log-probabilities for each of the label predictions and store them in the variables: **k_prob** (for ketone), **c_prob** (for carboxylic acid), and **o_prob** (for other carbonyl).

In [ ]:
# run this code to get the probabilities for each label prediction
k_prob=rf_ketone.predict_proba(X_test)[:,1]
c_prob=rf_cbxl.predict_proba(X_test)[:,1]
o_prob=rf_other.predict_proba(X_test)[:,1]

## Multiclass Classification
Now that you have the probabilities of each label prediction, you can determine the most probable label as the one with the highest probability. For example, if the probabilities are:
- Ketone: 0.20
- Carboxylic acid: 0.79
- Other carbonyl: 0.15

...then the final prediction would be **carboxylic acid**. (Note that the probabilities do not have to sum to 1 here, as they are probabilities for each independent yes-no prediction.)

Run the code block below to determine the most probable prediction and display a DataFrame comparing the probabilities, the prediction, and the actual label (i.e., the "truth").

In [ ]:
# get the molecule names
molName=test.iloc[:,0]
# create a DataFrame with the molecule names and the three prediction probabilities
d = {'molecules':molName,'ketone':k_prob, 'carboxylic_acid':c_prob,'other':o_prob}
result = pd.DataFrame(data=d)
# determine the final predicted label for each molecule as the label with the highest probability for each molecule
result['predict']=result.iloc[:,1:].idxmax(axis=1)
# get the truth for each molecule by looking at the labels in the test data set
result['truth']=test.iloc[:,2:5].idxmax(axis=1)
# display the table
result

Although there is only one way for a prediction to be correct, there are multiple ways for a prediction to be incorrect here because there are multiple labels.
Run the code block below to generate a table showing the different combinations of correct and incorrect predictions.

In [ ]:
# generate a table showing the different types of correct and incorrect predictions
overall=result.groupby(["predict", "truth"]).size().reset_index(name="Count")
overall

## Confusion Matrix
These results can be presented more clearly as a confusion matrix, a table in which there is a row for each true label and a column for each predicted label. (But to create more *confusion*, sometimes these matrices are presented with the rows and columns swapped.)
- [Wikipedia](https://en.wikipedia.org/wiki/Confusion_matrix)
- [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) documentation

The code block below generates a confusion matrix for this multiclass classification. It uses the Python library [seaborn](https://seaborn.pydata.org/) for data visualization.

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
labels = ['other','carboxylic_acid','ketone']
matrix = confusion_matrix(result['truth'], result['predict'],labels)
df = pd.DataFrame(matrix)
df.columns = labels
sn.heatmap(df, yticklabels = labels, annot = True)
plt.xlabel('Predicted') 
plt.ylabel('Actual')

# Assessing Overall Model Performance
To determine the overall performance of the model, we can calculate an overall accuracy score, equal to the total number of correct predictions divided by the total number of predictions. We can also determine accuracy scores for each of the three classes. For example, the ketone class accuracy score would be the total number of correct ketone predictions divided by the total number of true ketones in the test data.
Run the code blocks below to determine and display these accuracy scores.

In [ ]:
# define a function to calculate the class accuracy
def classAcc(className):
  filter1=overall['truth']==className
  filter2=overall['predict']==className
  a=sum(overall.where(filter1 &filter2).dropna()['Count'])
  b=sum(overall.where(overall['truth']==className).dropna()['Count'])
  return np.round(a/b,2)

In [ ]:
# calculate the overall accuracy
ov_acc=np.round(sum(overall.where(overall['predict']==overall['truth']).dropna()['Count'])/sum(overall['Count']),4)
# calculate the class accuracy for each of the three classes
k_acc=classAcc('ketone')
c_acc=classAcc('carboxylic_acid')
o_acc=classAcc('other')

In [ ]:
# make and display a DataFrame containing all the accuracies
perform={'Model':['Random Forest'],'overall':ov_acc,'class ketone':k_acc,'class carboxylic_acid': c_acc,'class other':o_acc}
final_acc=pd.DataFrame(data=perform,index=[0])
final_acc